# **GenoVarDis@IberLEF2024: Automatic Genomic Variants and Related Diseases using Named Entity Recognition with Large Language Models**

## Autor: Víctor Manuel Oliveros Villena


In [ ]:
# Esta es la ruta a la raíz de nuestro Drive.
# Si se prefiere otra ruta, añadir los directorios pertinentes a continuación de esta.
path_drive = '/content/drive/MyDrive'

### **Configuración del entorno**

In [ ]:
!pip install gliner transformers beartype

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 11.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cac

In [ ]:
import numpy as np
import torch
import sys
from google.colab import drive
import pandas as pd
from gliner import GLiNER
from transformers import BasicTokenizer

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## **Preprocesado**

In [ ]:
sys.path.append(path_drive + "/GenoVarDis/Prototypes")
from trainer import GlinerTrainer

In [ ]:
# Ruta al archivo TSV en Google Drive
path_train_text = path_drive + '/GenoVarDis/Data/train_text.tsv'
path_train_annotation = path_drive + '/GenoVarDis/Data/train_annotation.tsv'
path_dev_text = path_drive + '/GenoVarDis/Data/dev_text.tsv'
path_dev_annotation = path_drive + '/GenoVarDis/Data/dev_annotation.tsv'

# Leer el archivo TSV utilizando pandas
train_text = pd.read_csv(path_train_text, sep='\t')
train_annotation = pd.read_csv(path_train_annotation, sep='\t')
dev_text = pd.read_csv(path_dev_text, sep='\t')
dev_annotation = pd.read_csv(path_dev_annotation, sep='\t')

### **Preprocesado para adaptar el formato de nuestros ejemplos al empleado por modelos GLiNER**

In [ ]:
basic_tokenizer = BasicTokenizer()
data = np.array([], dtype=dict)

def tokenize_and_set_ids(example, type_set='train'):
  """
  Tokeniza el texto y asigna IDs y etiquetas NER basadas en anotaciones para un ejemplo dado.

  Args:
      example (dict): Un diccionario que contiene un caso clinico.
      type_set (str): Indica si el ejemplo es parte del conjunto de entrenamiento ('train') o validacion ('dev').

  Returns:
      pd.DataFrame: DataFrame actualizado con los tokens y etiquetas NER para el ejemplo.
  """
  global data
  indices = set()
  result = dict()

  pmid = example['pmid']    # Pmid del caso clinico
  text = example['text']    # Texto del caso clinico
  tokens = np.array(basic_tokenizer.tokenize(text)) # Tokeniza el texto
  tags = []

  # Toma de datos del archivo correspondiente segun la opcion escogida
  if type_set == 'dev':
    sort_df = dev_annotation[dev_annotation['pmid'] == pmid].sort_values(by='offset1')
  else:
    sort_df = train_annotation[train_annotation['pmid'] == pmid].sort_values(by='offset1')
  labels = sort_df['label'].values.astype(str)    # Tipos de entidades
  spans = sort_df['span'].values.astype(str)      # Entidades

  index = 0
  # Por cada entidad...
  for l, d in zip(labels, spans):
    span_split = np.array(basic_tokenizer.tokenize(d)) # Tokeniza la entidad
    # Busca todas las apariciones de la entidad en el caso clinico
    index = np.where(tokens == span_split[0])[0]
    found = False
    # Si la entidad solo es una palabra...
    if len(span_split) == 1:
      i = 0
      # Mientras que no encontremos el indice de aparicion correcto...
      while not found and i < len(index):
        # Si el indice es valido (no fue escogido previamente)...
        if index[i] not in indices and (len(indices) == 0 or index[i] > max(indices)):
          # Lo guardamos en la lista de indices asignados
          indices.add(index[i])
          # Asignamos el tipo de entidad segun el modelo GLiNER
          tags.append([index[i], index[i], l])
          # Marcamos que hemos encontrado un indice
          found = True
        i += 1
    # Si la entidad contiene varias palabras...
    else:
      k = 0
      # Mientras que no encontramos el indice de aparicion correcto...
      while not found and k < len(index):
        i = index[k]
        # Si el indice es valido (no fue escogido previamente)...
        if i not in indices and (len(indices) == 0 or i > max(indices)):
          # Si la totalidad de la entidad coincide...
          if np.array_equal(tokens[i:i+len(span_split)], span_split):
            # Marcamos los indices correspondientes como asignados
            indices.update(np.arange(i, i+len(span_split)))
            # Asignamos el tipo de entidad segun el modelo GLiNER
            tags.append([i, i+len(span_split)-1, l])
            found = True
        k += 1
  # Guardamos el resultado final en un formato compatible con los modelos GLiNER
  result['tokenized_text'] = tokens.tolist()
  result['ner'] = tags

  # Añadimos el resultado a nuestro dataframe
  data = np.append(data, result)

  return data.tolist()

In [ ]:
# Aplicamos la funcion anterior a la totalidad del conjunto de entrenamiento
train_text.apply(tokenize_and_set_ids, axis=1)

0      [{'tokenized_text': ['12672033', '|', 't', '|'...
1      [{'tokenized_text': ['12672033', '|', 't', '|'...
2      [{'tokenized_text': ['12672033', '|', 't', '|'...
3      [{'tokenized_text': ['12672033', '|', 't', '|'...
4      [{'tokenized_text': ['12672033', '|', 't', '|'...
                             ...                        
422    [{'tokenized_text': ['12672033', '|', 't', '|'...
423    [{'tokenized_text': ['12672033', '|', 't', '|'...
424    [{'tokenized_text': ['12672033', '|', 't', '|'...
425    [{'tokenized_text': ['12672033', '|', 't', '|'...
426    [{'tokenized_text': ['12672033', '|', 't', '|'...
Length: 427, dtype: object

In [ ]:
# Aplicamos la funcion anterior a la totalidad del conjunto de validacion
dev_text.apply(lambda row: tokenize_and_set_ids(row, type_set='dev'), axis=1)

0     [{'tokenized_text': ['12672033', '|', 't', '|'...
1     [{'tokenized_text': ['12672033', '|', 't', '|'...
2     [{'tokenized_text': ['12672033', '|', 't', '|'...
3     [{'tokenized_text': ['12672033', '|', 't', '|'...
4     [{'tokenized_text': ['12672033', '|', 't', '|'...
                            ...                        
65    [{'tokenized_text': ['12672033', '|', 't', '|'...
66    [{'tokenized_text': ['12672033', '|', 't', '|'...
67    [{'tokenized_text': ['12672033', '|', 't', '|'...
68    [{'tokenized_text': ['12672033', '|', 't', '|'...
69    [{'tokenized_text': ['12672033', '|', 't', '|'...
Length: 70, dtype: object

In [ ]:
# Total de ejemplos: Ejemplos de entrenamiento + Ejemplos de validacion
print(len(data))

497


In [ ]:
# Pasamos los datos a lista primitiva de Python para ser compatible con Trainer.py
data = data.tolist()

## **Entrenamiento del fine-tuning de GLiNER Medium**

In [ ]:
# Cargamos el modelo GLiNER concreto que vamos a emplear, el Medium
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/781M [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

In [ ]:
# Establecemos el dispositivo donde ejecutar el modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Diccionario con informacion sobre el conjunto de validacion y tipos de entidades
eval_data = {
    "entity_types": ["Gene", "Disease", "DNAMutation", "SNP", "DNAAllele", "NucleotideChange-BaseChange", "OtherMutation", "Transcript"],
    "samples": data[:int(len(data)*0.1)]
}

# Establecemos los hiperparametros del modelo
trainer = GlinerTrainer(model,
                        train_data = data[int(len(data)*0.1):],
                        batch_size = 2,
                        grad_accum_every = 16,
                        lr_encoder = 1e-5,
                        lr_others = 5e-5,
                        freeze_token_rep = False,
                        val_every_step = 224,
                        val_data = eval_data,
                        checkpoint_every_epoch = 10, # Or checkpoint_every_step if you use steps
                        max_types=25,
                        max_len=700,
)

torch.cuda.empty_cache()

trainer.train(num_epochs=20)

Training for 4480 steps which is 20 epochs.


Epoch 1/20: 100%|██████████| 224/224 [02:02<00:00,  1.83it/s, average_loss=104, loss=96.1, step=224]


Step=224
P: 68.34%	R: 55.95%	F1: 61.53%

Epoch 1 average loss: 104.48708699430738


Epoch 2/20: 100%|██████████| 224/224 [02:00<00:00,  1.85it/s, average_loss=79.2, loss=96.3, step=448]


Step=448
P: 68.43%	R: 61.96%	F1: 65.03%

Epoch 2 average loss: 79.23309877940586


...

Epoch 19/20: 100%|██████████| 224/224 [02:01<00:00,  1.84it/s, average_loss=28.3, loss=34.2, step=4256]


Step=4256
P: 75.60%	R: 73.08%	F1: 74.32%

Epoch 19 average loss: 28.307048923202924


Epoch 20/20: 100%|██████████| 224/224 [02:01<00:00,  1.84it/s, average_loss=28.8, loss=55.7, step=4480]

Step=4480
P: 75.60%	R: 73.08%	F1: 74.32%

Epoch 20 average loss: 28.76741732444082


28.76741732444082

In [ ]:
# Guardamos el modelo
trainer.model.save_pretrained(path_drive + "/GenoVarDis/Logs/GLiNERMedium_Trainer")